# Testing
Adjust as needed for project versions and testing

## Imports

In [ ]:
import numpy as np
import pandas as pd
import librosa as lb
import soundfile as sf
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.backend import clear_session

import os
import glob

## Parameters

In [ ]:
effect = "SD-1"
projVer = "DS340"
numModels = 36

sampleRate = 44100
channels = 1

## Functions

In [ ]:
def esrLoss(true, pred):
    return tf.reduce_sum(tf.pow(true - pred,2), axis = 0) / tf.reduce_sum(tf.pow(true, 2), axis = 0) + 1e-50

#Prepares test audio for prediction
def predictionPrep(audio, samples):
    chunks = []
    for i in range(len(audio) - samples):
        chunks.append(audio[i:i+samples])
    return np.array(chunks)

#Evaluates test audio
#Due to how sequences are batched, there is an offset
def evaluateNew(model, xTest, yTest, offset):
    yPred = model.predict(predictionPrep(xTest, offset))
    return esrLoss(yTest[offset:], yPred.squeeze()), yPred


## Execute

In [ ]:
#THIS DOES NOT NEED TO BE RUN AGAIN IF MODEL RESULTS ARE STORED IN modelResults.pkl
cols = ["modelNum", "lstmUnits", "learningRate", "context", "gain"]
modelResults = pd.DataFrame(columns = cols)

for i in range(numModels):
    clear_session()
    directory = f"../Models/{effect}/{projVer}/Model_{i}"
    h5 = glob.glob(f"{directory}/*.h5")[0]
    model = load_model(h5, custom_objects={"esrLoss": esrLoss})
        
    modelAttributes = h5.split("_")

    lvl = modelAttributes[2]
    #print(lvl)
    units = int(modelAttributes[3])
    rate = float(modelAttributes[4])
    context = int(modelAttributes[5])

    #print(f"{lvl} {units} {rate} {context}")

    modelRow = {

        "modelNum" : i,
        "lstmUnits" : units,
        "learningRate" : rate,
        "context" : context,
        "gain" : lvl

    }
        
    #did some organizational messing around here so all 24 wouldn't have the be evaluated
    #ill fix the organization later

    dataPath = f"../Data/{effect}/{projVer}/Test/{lvl}"
    for _, _, files in os.walk(f"{dataPath}/Dry"):
        for file in files:
            fileName = file[2:]
            dry, _ = lb.load(f"{dataPath}/Dry/d_" + fileName, sr=sampleRate, mono=True)
            wet, _ = lb.load(f"{dataPath}/Wet/w_" + fileName, sr=sampleRate, mono=True)
            #change evalutateNew to return a float not a tensor or something
            loss, output = evaluateNew(model, dry, wet, context)
            os.makedirs(f"{directory}/Output", exist_ok=True)
            #or use PCM_16
            sf.write(f"{directory}/Output/o_{fileName}", output, sampleRate, subtype = "PCM_24")
            #creates new columns for the songs
            if fileName not in modelResults.columns:
                modelResults[fileName] = None 
            modelRow[fileName] = float(loss.numpy())
            #print(fileName, float(loss.numpy()))
    #check if this ordering is right later
    modelResults = pd.concat([modelResults, pd.DataFrame([modelRow])], ignore_index = True)
    #saves the results

    outputDir = f"../ModelResults/{effect}/{projVer}"
    os.makedirs(outputDir, exist_ok=True)
    modelResults.to_pickle(f"{outputDir}/{effect}_{projVer}_modelResults.pkl")
    #print(modelResults)
